In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import sqlite3
import numpy as np

# Tách dữ liệu riêng của CSKCB 93005



In [3]:
# chỉnh sửa tên csckb nếu muốn thay đổi tại cskcb nào ???
CSCKB = 'CSKCB_93002 ';

In [4]:
# loại câu lệnh gì khi thao tác với CSDL
syntax = 'CREATE TABLE ';

In [5]:
# dữ liệu cần lấy khi thực thi câu lệnh SQL.
sql_query = (r'as '
            + ' Select MA_CHA,MA_CSKCB,NAM_QT,KY_QT,XML1_ID,TEN_CHOT,MA_THE,HO_TEN,NGAY_SINH,GIOI_TINH,NGAY_VAO,NGAY_RA,MA_BENH,MA_BENHKHAC,MA_LOAIKCB,LOAI_BN,'
            +' T_TONGCHI,T_XN,T_CDHA,T_THUOC,T_MAU,T_PTTT,T_VTYT,T_DVKT_TYLE,T_THUOC_TYLE,T_VTYT_TYLE,T_KHAM,T_GIUONG,T_VCHUYEN,T_BNCCT,T_BNTT,T_BHTT,'
            +' T_XN_BHTT,T_CDHA_BHTT,T_THUOC_BHTT,T_MAU_BHTT,T_PTTT_BHTT,T_VTYT_BHTT,T_DVKT_TYLE_BHTT,T_THUOC_TYLE_BHTT,T_VTYT_TYLE_BHTT,T_KHAM_BHTT,T_GIUONG_BHTT,T_VCHUYEN_BHTT,T_NGOAIDS,T_NGUONKHAC,'
            +' TINH_LUOT,TINH_TIEN,GIAM_DINH,T_XUATTOAN_TONGCHI,T_XUATTOAN_BHTT,LYDO_XT,MA_BN,DIA_CHI,MA_DKBD,GT_THE_TU,GT_THE_DEN,MA_KHUVUC,MA_LYDO_VVIEN,MA_NOI_CHUYEN,SO_NGAY_DTRI,KET_QUA_DTRI,TINH_TRANG_RV,MA_KHOA,T_TRU_CCG,T_BHTT_SAU_CCG,MA_LOAIKCB_GOC,TINH_LUOT_GOC,LUOT_KBT_XAC_NHAN,TACH_LUOT_XAC_NHAN,MA_LOAI_KCB_CN'
            +' from gsgd_bhyt7980b_final')

In [6]:
# điều kiện xử lý SQL
condition = ' where MA_CHA = 93002 and KY_QT BETWEEN 202001 and 202112'
# xử lý tạo bảng
query = syntax + CSCKB + sql_query  + condition

In [7]:
def updateSqliteTable(query):
    try:       
        con = sqlite3.connect(r'E:\Cn\93_20220329_202001_202112_bao_cao_gsgd_bhyt7980b_final.db')
        cursor = con.cursor()  
        sql_update_query = query   
        cursor.execute(sql_update_query)        
        con.commit() 
        return cursor.rowcount
        #print("Number row update " + str(cursor.rowcount))
        cursor.close()
    except sqlite3.Error as error:
        print("Failed to update sqlite table", error)
    finally:
        if con:
            con.close()      

In [8]:
updateSqliteTable(query)
#print("Create table" + query)

-1

In [9]:
syntax = 'ALTER TABLE ';

In [10]:
sql_query = 'ADD COLUMN LUOT_DUNG INT'
# xử lý thêm cột lượt đúng
query = syntax + CSCKB + sql_query
updateSqliteTable(query)

-1

In [11]:
sql_query = 'ADD COLUMN BENH_DUNG TEXT'
# xử lý thêm cột bệnh đúng
query = syntax + CSCKB + sql_query
updateSqliteTable(query)

-1

In [12]:
sql_query = 'ADD COLUMN MA_LOAI_KCB_DUNG INT'
# xử lý thêm cột bệnh đúng
query = syntax + CSCKB + sql_query
updateSqliteTable(query)

-1

In [13]:
# xử lý trường hợp giám định 3 loại xuất toán toàn bộ
syntax = 'UPDATE '
sql_query = (r'SET LUOT_DUNG = 0, BENH_DUNG = "GD3" WHERE TINH_LUOT = 0 AND TINH_TIEN = 0')
query = syntax + CSCKB + sql_query
countrow = 0;
countrow += updateSqliteTable(query)
print("Number row changed : " + str(countrow))

Number row changed : 262


In [14]:
# các đối tượng đúng còn lại
sql_query = (r'SET LUOT_DUNG = 1, BENH_DUNG = MA_BENH WHERE TINH_TIEN = 1 AND T_TONGCHI != T_KHAM ')
query = syntax + CSCKB + sql_query
countrow = 0;
countrow += updateSqliteTable(query)
print("Number row changed : " + str(countrow))

Number row changed : 192662


In [15]:
# xử lý các đối tượng chỉ có tiền công khám KHB
sql_query = (r'SET LUOT_DUNG = 1, BENH_DUNG = "KHB" WHERE  TINH_TIEN = 1 and T_TONGCHI = T_KHAM ')
query = syntax + CSCKB + sql_query
countrow = 0;
countrow += updateSqliteTable(query)
print("Number row changed : " + str(countrow))

Number row changed : 5394


In [16]:
# cập nhật lượt đúng bằng 0 cho các trường hợp ghép lượt và các trường hợp tách lượt do trùng, tách đợt điều trị
sql_query = (r'SET LUOT_DUNG = 0, BENH_DUNG = MA_BENH  WHERE TACH_LUOT_XAC_NHAN = 1 OR LUOT_KBT_XAC_NHAN = 1 ')
query = syntax + CSCKB + sql_query
countrow = 0;
countrow += updateSqliteTable(query)
print("Number row changed : " + str(countrow))

Number row changed : 2963


In [17]:
# cập nhật lại loại KCB cho Cn
sql_query = (r'SET MA_LOAI_KCB_DUNG = MA_LOAI_KCB_CN ')
query = syntax + CSCKB + sql_query
countrow = 0;
countrow += updateSqliteTable(query)
print("Number row changed : " + str(countrow))

Number row changed : 198318


# Danh sách dữ liệu phát sinh năm 2020

In [6]:
syntax = 'SELECT'
# chỉnh sửa tên csckb nếu muốn thay đổi tại cskcb nào ???
CSCKB = 'CSKCB_93002 '; 
# dữ liệu cần lấy khi thực thi câu lệnh SQL.
sql_query = (r' CASE WHEN MA_LOAI_KCB_DUNG = 1 THEN "Khám bệnh"'
		+ ' WHEN MA_LOAI_KCB_DUNG = 2 THEN "Điều trị ngoại trú"'
		+ ' WHEN MA_LOAI_KCB_DUNG = 3 THEN "Điều trị nội trú"'
		+ ' WHEN MA_LOAI_KCB_DUNG = 4 THEN "Giường lưu TYT"'
        + ' WHEN MA_LOAI_KCB_DUNG = 5 THEN "Nội trú ban ngày"'
		+ ' ELSE "Khác"'	
		+'  END As "LOAI_KCB", * FROM ')

condition = "WHERE MA_CHA = 93002 AND KY_QT BETWEEN 202001 and 202012 AND SUBSTR(MA_THE,1,2) not in ('QN','CA','CY') AND BENH_DUNG != 'GD3'"

query = syntax + sql_query + CSCKB + condition

In [7]:
#query

In [8]:
con = sqlite3.connect(r'D:\DATACSDL\CSDL\DataCn\NAM2020-2021\93_20220329_202001_202112_bao_cao_gsgd_bhyt7980b_final.db')
Read_Data_2020 = pd.read_sql_query(query,con);
# xử lý dữ liệu bỏ ra các trường hợp giám định xuất toán toàn bộ và các đối tượng nghị định số 70 (QN-CA-CY)
List2020 = Read_Data_2020[((Read_Data_2020['MA_THE'].astype(str).str.slice(0, 2) != 'QN') | (Read_Data_2020['MA_THE'].astype(str).str.slice(0, 2) != 'CA') | (Read_Data_2020['MA_THE'].astype(str).str.slice(0, 2) != 'CY'))]
print("Số lượng dữ liệu được sử dụng" + str(List2020.shape))

Số lượng dữ liệu được sử dụng(112290, 75)


In [9]:
#List2020.shape
#List2020.info()

# Danh sách dữ liệu phát sinh năm 2021

In [10]:
syntax = 'SELECT'
# chỉnh sửa tên csckb nếu muốn thay đổi tại cskcb nào ???
CSCKB = 'CSKCB_93005'; 
# dữ liệu cần lấy khi thực thi câu lệnh SQL.
sql_query = (r' CASE WHEN MA_LOAI_KCB_DUNG = 1 THEN "Khám bệnh"'
		+ ' WHEN MA_LOAI_KCB_DUNG = 2 THEN "Điều trị ngoại trú"'
		+ ' WHEN MA_LOAI_KCB_DUNG = 3 THEN "Điều trị nội trú"'
		+ ' WHEN MA_LOAI_KCB_DUNG = 4 THEN "Giường lưu TYT"'
        + ' WHEN MA_LOAI_KCB_DUNG = 5 THEN "Nội trú ban ngày"'
		+ ' ELSE "Khác"'	
		+'  END As "LOAI_KCB", * FROM ')
condition = " WHERE MA_CHA = 93005 AND KY_QT BETWEEN 202101 and 202112 AND SUBSTR(MA_THE,1,2) not in ('QN','CA','CY') AND BENH_DUNG != 'GD3' "

query = syntax + sql_query + CSCKB + condition

In [11]:
con = sqlite3.connect(r'D:\DATACSDL\CSDL\DataCn\NAM2020-2021\93_20220329_202001_202112_bao_cao_gsgd_bhyt7980b_final.db')
Read_Data_2021 = pd.read_sql_query(query,con);
# xử lý dữ liệu bỏ ra các trường hợp giám định xuất toán toàn bộ và các đối tượng nghị định số 70 (QN-CA-CY)
List2021 = Read_Data_2021[((Read_Data_2021['MA_THE'].astype(str).str.slice(0, 2) != 'QN') | (Read_Data_2021['MA_THE'].astype(str).str.slice(0, 2) != 'CA') | (Read_Data_2021['MA_THE'].astype(str).str.slice(0, 2) != 'CY'))]
print("Số lượng dữ liệu được sử dụng" + str(List2021.shape))

Số lượng dữ liệu được sử dụng(85529, 75)


In [12]:
# List2021.shape
# List2021.info()

# Xử lý dữ liệu 2 năm 

In [13]:
result2020 = List2020[["XML1_ID", "MA_THE","MA_CSKCB","NAM_QT","KY_QT","T_VCHUYEN_BHTT","T_BHTT_SAU_CCG","LOAI_KCB","LUOT_DUNG","BENH_DUNG"]].copy()
# xử lý lấy mã bệnh
result2020.loc[result2020['BENH_DUNG'].astype(str).str.slice(0, 1) == 'U',"BENH_DUNG"] = result2020['BENH_DUNG'].astype(str).str.slice(0, 7)
result2020.loc[result2020['BENH_DUNG'].astype(str).str.slice(0, 1) != 'U',"BENH_DUNG"] = result2020['BENH_DUNG'].astype(str).str.slice(0, 3)
#result2020['BENH_DUNG'] =  result2020['BENH_DUNG'].astype(str).str.slice(0, 3)

In [14]:
result2021 = List2021[["XML1_ID", "MA_THE","MA_CSKCB","NAM_QT","KY_QT","T_VCHUYEN_BHTT","T_BHTT_SAU_CCG","LOAI_KCB","LUOT_DUNG","BENH_DUNG"]].copy()
# xử lý lấy mã bệnh
result2021.loc[result2021['BENH_DUNG'].astype(str).str.slice(0, 1) == 'U',"BENH_DUNG"] = result2021['BENH_DUNG'].astype(str).str.slice(0, 7)
result2021.loc[result2021['BENH_DUNG'].astype(str).str.slice(0, 1) != 'U',"BENH_DUNG"] = result2021['BENH_DUNG'].astype(str).str.slice(0, 3)
#result2021['BENH_DUNG'] =  result2021['BENH_DUNG'].astype(str).str.slice(0, 3)

In [15]:
#result2021
#frames = [result2020, result2021]

In [16]:
#CSCKB93016_C7Data = pd.concat(frames)
#Data_2020 = result2020.groupby(['MA_CSKCB','TUYEN','LOAI_KCB','BENH_DUNG']).agg({'LUOT_DUNG':'sum','T_BHTT_SAU_CCG':'sum' ,'T_VCHUYEN_BHTT' : 'sum'}).reset_index()
Data_2020 = result2020.groupby(['MA_CSKCB','LOAI_KCB','BENH_DUNG']).agg({'LUOT_DUNG':'sum','T_BHTT_SAU_CCG':'sum' ,'T_VCHUYEN_BHTT' : 'sum'}).reset_index()
Data_2020['CPBQ'] =  (Data_2020["T_BHTT_SAU_CCG"] - Data_2020["T_VCHUYEN_BHTT"]) / Data_2020["LUOT_DUNG"]
Data_2020['T_BHTT_SAU_CCG'] = Data_2020['T_BHTT_SAU_CCG'] - Data_2020['T_VCHUYEN_BHTT'] 
#Data_2020.reset_index()
# Data_2020['MA_CSKCB'] = Data_2020['MA_CSKCB'].apply(np.int64)
# Data_2020['MA_LOAIKCB'] = Data_2020['MA_LOAIKCB'].apply(np.int64)
#Data_2020.info()


In [17]:
#CSCKB93016_C7Data = pd.concat(frames)
Data_2021 = result2021.groupby(['MA_CSKCB','LOAI_KCB','BENH_DUNG']).agg({'LUOT_DUNG':'sum','T_BHTT_SAU_CCG':'sum' ,'T_VCHUYEN_BHTT' : 'sum'}).reset_index()
Data_2021['CPBQ'] =  (Data_2021["T_BHTT_SAU_CCG"] - Data_2021["T_VCHUYEN_BHTT"]) / Data_2021["LUOT_DUNG"]
Data_2021['T_BHTT_SAU_CCG'] = Data_2021['T_BHTT_SAU_CCG'] - Data_2021['T_VCHUYEN_BHTT'] 
#Data_2021.info()

In [18]:
# outer là điều kiện lấy đảm bảo đầy đủ dữ liệu của 2 năm
CSCKB93005_C7Data = pd.merge(Data_2020, Data_2021,  how='outer', left_on=['MA_CSKCB','LOAI_KCB','BENH_DUNG'], right_on = ['MA_CSKCB','LOAI_KCB','BENH_DUNG'],suffixes=('_KYTRUOC', '_KYHIENTAI'))


In [19]:
CSCKB93005_C7Data = CSCKB93005_C7Data.replace(np.nan, 0)
CSCKB93005_C7Data.loc[CSCKB93005_C7Data["LUOT_DUNG_KYTRUOC"] == 0, "TANG/GIAM"] = CSCKB93005_C7Data["T_BHTT_SAU_CCG_KYHIENTAI"]
CSCKB93005_C7Data.loc[CSCKB93005_C7Data["LUOT_DUNG_KYTRUOC"] != 0, "TANG/GIAM"] =  (CSCKB93005_C7Data["LUOT_DUNG_KYHIENTAI"] - CSCKB93005_C7Data["LUOT_DUNG_KYTRUOC"]) * ((CSCKB93005_C7Data["T_BHTT_SAU_CCG_KYTRUOC"] - CSCKB93005_C7Data["T_VCHUYEN_BHTT_KYTRUOC"]) / CSCKB93005_C7Data["LUOT_DUNG_KYTRUOC"]) * 1
CSCKB93005_C7Data['BQ_CHENHLECH'] = CSCKB93005_C7Data['CPBQ_KYHIENTAI']  - CSCKB93005_C7Data['CPBQ_KYTRUOC']
CSCKB93005_C7Data['CHIPHI_CHENHLECH'] = CSCKB93005_C7Data['T_BHTT_SAU_CCG_KYHIENTAI']  - CSCKB93005_C7Data['T_BHTT_SAU_CCG_KYTRUOC']


In [20]:
#CSCKB93102_C7Data

In [21]:
pd.options.display.float_format = '{:,.2f}'.format
CSCKB93005_C7Data.groupby("MA_CSKCB")['TANG/GIAM'].sum()


MA_CSKCB
93005   -4,348,105,145.34
93026      -20,149,860.45
93027        5,713,153.93
93028      -19,180,908.00
93029      -29,417,931.78
93030       10,341,381.43
93031      -40,054,939.18
93032      -11,744,208.74
93033       15,964,161.93
93087        9,355,887.56
Name: TANG/GIAM, dtype: float64

In [22]:
print('Chi phí C7 của CSCKB:','{0:,.2f}'.format(CSCKB93005_C7Data['TANG/GIAM'].sum()))

Chi phí C7 của CSCKB: -4,427,278,408.64


In [23]:
CSCKB93005_C7Data.to_excel(r'D:\SOLIEU\THAMDINHCn\CSKCB93005_C7KCB.xlsx')